Gökçe Şahin
090190306
sahingo19@itu.edu.tr

In [378]:
import pandas as pd
import wbgapi as wb
import numpy as np
import statsmodels.api as sm
import yfinance as yf

from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statsmodels.tools as tools

from statsmodels.formula.api import logit
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Question 1

For this question use the World Bank Data for Turkey for the following indicators. Use [wbgapi](https://pypi.org/project/wbgapi/) for getting the data.

* [Literacy rate, adult female (SE.ADT.LITR.FE.ZS)](https://data.worldbank.org/indicator/SE.ADT.LITR.FE.ZS)
* [Labor force, female (SL.TLF.TOTL.FE.ZS)](https://data.worldbank.org/indicator/SL.TLF.TOTL.FE.ZS)
* [Poverty headcount ratio at national poverty lines (SI.POV.NAHC)](https://data.worldbank.org/indicator/SI.POV.NAHC)
* [Current health expenditure per capita (SH.XPD.CHEX.PC.CD)](https://data.worldbank.org/indicator/SH.XPD.CHEX.PC.CD)
* [GDP per capita (NY.GDP.PCAP.CD)](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
* [Mortality rate, under-5 (SH.DYN.MORT)](https://data.worldbank.org/indicator/SH.DYN.MORT)


Using the [statsmodels](https://www.statsmodels.org/stable/index.html) library write the best linear regression model using child mortality as the dependent variable while the rest are considered as independent variables. Pay particular attention to the fact that the order of the variables put into the model significantly impacts the performance of the model. Choose the best model by considering

* with the minimum number of variables and their interactions,
* with the optimal ordering of the independent variables and their interactions,
* $R^2$-score of the model,
* statistical significance of the model coefficients,
* ANOVA analysis of the model.


In [379]:
def extract(df, cntry, name):
    tmp = df[['time','value']][df['economy'] == cntry]
    tmp.index = tmp.time
    del tmp['time']
    tmp.columns = [[name]]
    return tmp

In [380]:
def vars_mor(cntry, dfs, vars):
    arr = [extract(dfs[i], cntry, vars[i]) for i in range(len(vars))]
    res = arr[0].join(arr[1:6])
    res.dropna(inplace=True)
    
    return res

In [381]:
var_apis = {
    'literacy': 'SE.ADT.LITR.FE.ZS',
    'labor': 'SL.TLF.TOTL.FE.ZS',
    'poverty': 'SI.POV.NAHC',
    'health': 'SH.XPD.CHEX.PC.CD',
    'GDP': 'NY.GDP.PCAP.CD',
    'mortality': 'SH.DYN.MORT'
}


In [382]:
vars = ['literacy', 'labor', 'mortality']
# independent variables + dependent variable (mortality)

In [383]:
dfs = [pd.DataFrame(list(wb.data.fetch(api))) for api in [var_apis[var] for var in vars]]

APIResponseError: APIError: JSON decoding error (https://api.worldbank.org/v2/en/sources/2/series/SE.ADT.LITR.FE.ZS/country/all/time/all?per_page=1000&page=12&format=json)

In [ ]:
res = vars_mor('TUR', dfs, vars)
model = ols('mortality ~ literacy + labor', data=res).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     252.1
Date:                Thu, 03 Nov 2022   Prob (F-statistic):           1.58e-10
Time:                        23:52:45   Log-Likelihood:                -33.794
No. Observations:                  15   AIC:                             73.59
Df Residuals:                      12   BIC:                             75.71
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    147.9560      8.876     16.669      0.0

C:\Users\sahin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# Question 2

For this question use Yahoo's Finance API for the following tickers:

* Gold futures (GC=F)
* Silver futures (SI=F)
* Copper futures (HG=F)
* Platinum futures (PL=F)

1. Write the best linear regression model that explains gold futures closing prices in terms of opening prices of gold, silver, copper, and platinum futures.
2. Repeat the same for silver, copper and platinum prices.
3. Compare the models you obtained in Steps 1 and 2. Which model is better? How do you decide? Explain.

In [ ]:
def tickers_openning(x, ftickers, fticker_names):
    tmp = {}
    tmp['GCF_o'] = ftickers[0]['Open']
    tmp['SIF_o'] = ftickers[1]['Open']
    tmp['HGF_o'] = ftickers[2]['Open']
    tmp['PLF_o'] = ftickers[3]['Open']
    tmp[x[:-2]+x[-1]+'_c'] = ftickers[fticker_names.index(x)]['Close']

    data = pd.DataFrame(tmp).dropna()

    return data

In [ ]:
fticker_names = ['GC=F', 'SI=F', 'HG=F', 'PL=F']
ftickers = [yf.download(ticker) for ticker in fticker_names]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
[gcfc, sifc, hgfc, plfc] = [tickers_openning(x, ftickers, fticker_names) for x in fticker_names]
gcfc


,GCF_o,SIF_o,HGF_o,PLF_o,GCF_c
Date,,,,,
2000-08-30,273.899994,4.950000,0.8790,593.900024,273.899994
2000-08-31,274.799988,4.920000,0.8850,589.000000,278.299988
2000-09-01,277.000000,5.035000,0.8780,588.000000,277.000000
2000-09-05,275.799988,4.990000,0.8960,602.000000,275.799988
2000-09-06,274.200012,5.000000,0.9050,603.000000,274.200012
...,...,...,...,...,...
2022-10-28,1661.000000,19.500000,3.5375,958.400024,1639.599976
2022-10-31,1641.800049,19.200001,3.4095,939.400024,1635.900024
2022-11-01,1630.800049,19.125000,3.4945,959.799988,1645.000000


### Linear regression models that explains GC = F

In [ ]:
model = ols('GCF_c ~ SIF_o + HGF_o + PLF_o', data=gcfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  GCF_c   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                 1.167e+04
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        00:04:22   Log-Likelihood:                -32171.
No. Observations:                4863   AIC:                         6.435e+04
Df Residuals:                    4859   BIC:                         6.438e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    431.4354      8.454     51.030      0.000     414.861     448.010
SIF_o         49.1478      0.617     79.611      0.000      47.938      50.358
HGF_o        228.9932      4.671     49.030      0.000     219.837     238.150
PLF_o         -0.6944      0.011    -61.116      0.000      -0.717      -0.672
==============================================================================
Omnibus:                      375.758   Durbin-Watson:                   0.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              873.215
Skew:                          -0.478   Prob(JB):                    2.42e-190
Kurtosis:                       4.843   Cond. No.                     3.78e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.78e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains SI = F

In [ ]:
model = ols('SIF_c ~ GCF_o + HGF_o + PLF_o', data=sifc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  SIF_c   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                 1.431e+04
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        00:04:25   Log-Likelihood:                -11839.
No. Observations:                4863   AIC:                         2.369e+04
Df Residuals:                    4859   BIC:                         2.371e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.5911      0.129    -50.995      0.000      -6.845      -6.338
GCF_o          0.0115      0.000     79.577      0.000       0.011       0.012
HGF_o         -0.4496      0.087     -5.167      0.000      -0.620      -0.279
PLF_o          0.0109      0.000     63.802      0.000       0.011       0.011
==============================================================================
Omnibus:                     1298.959   Durbin-Watson:                   0.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6387.300
Skew:                           1.199   Prob(JB):                         0.00
Kurtosis:                       8.076   Cond. No.                     5.44e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.44e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains HG = F

In [ ]:
model = ols('HGF_c ~ GCF_o + SIF_o + PLF_o', data=hgfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  HGF_c   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     8059.
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        00:04:28   Log-Likelihood:                -3071.9
No. Observations:                4863   AIC:                             6152.
Df Residuals:                    4859   BIC:                             6178.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3248      0.026    -12.493      0.000      -0.376      -0.274
GCF_o          0.0015   2.96e-05     49.005      0.000       0.001       0.002
SIF_o         -0.0129      0.002     -5.453      0.000      -0.017      -0.008
PLF_o          0.0015   3.18e-05     46.063      0.000       0.001       0.002
==============================================================================
Omnibus:                      645.744   Durbin-Watson:                   0.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              956.449
Skew:                           0.974   Prob(JB):                    2.04e-208
Kurtosis:                       3.961   Cond. No.                     6.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains PL = F

In [ ]:
model = ols('PLF_c ~ GCF_o + SIF_o + HGF_o', data=plfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PLF_c   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     5762.
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        00:04:29   Log-Likelihood:                -31920.
No. Observations:                4863   AIC:                         6.385e+04
Df Residuals:                    4859   BIC:                         6.387e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    524.4117      6.522     80.406      0.000     511.625     537.198
GCF_o         -0.6274      0.010    -61.218      0.000      -0.647      -0.607
SIF_o         41.9352      0.657     63.791      0.000      40.646      43.224
HGF_o        207.9820      4.526     45.948      0.000     199.108     216.856
==============================================================================
Omnibus:                      247.090   Durbin-Watson:                   0.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              326.959
Skew:                           0.493   Prob(JB):                     1.00e-71
Kurtosis:                       3.801   Cond. No.                     3.48e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

***
Linear regression model that explains SI=F closing prices is better than the others because its R^2 value is larger.
***

# Question 3

1. Write a function that takes a ticker symbol and returns a pandas dataframe that for each day puts a 1 when the closing price is higher than the opening price, a 0 when the closing price is lower than the opening price.
2. Write the best logistic regression that predicts the time series you obtain from Step 1 for gold futures against the opening prices of gold, silver, copper, and platinum prices.
3. Repeat the same for silver, copper, and platinum prices.
4. Compare the models you obtained from Steps 2 and 3. Decide which is the best model, and explain your reasoning.
5. Does any of the models provide a good fit? Explain.

In [ ]:
def openning_closing(ticker):
    new_header = ticker[:-2]+ticker[-1]+'_co'

    tmp = yf.download(ticker)
    tmp[new_header] = 0
    tmp[new_header][tmp['Close'] > tmp['Open']] = 1

    tmp_co = pd.DataFrame(tmp[new_header])

    return tmp_co

In [ ]:
def ticker_co_df(ticker, tickers_df):
    df = pd.concat([tickers_df, openning_closing(ticker)], axis = 1)
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    del df['Date']
    return df


In [ ]:
def bootstrap(X, y, model):
    res = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)
        model.fit(X_train,y_train)
        res.append(model.score(X_test,y_test))
    tmp = sorted(res)[3:97]
    return (min(tmp),max(tmp))

In [ ]:
def print_model(df):
    X = df.iloc[:, :4]
    y = df.iloc[:, 5]

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

    model = LogisticRegression(max_iter=1500)
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    y_predict = model.predict(X_test)
    print(confusion_matrix(y_test, y_predict))

    print(bootstrap(X, y, model))


In [ ]:
fticker_names = ['GC=F', 'SI=F', 'HG=F', 'PL=F']
ftickers = [yf.download(ticker) for ticker in fticker_names]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
# I used tickers_openning() function from question-2
gcf_df = tickers_openning('GC=F', ftickers, fticker_names)
sif_df = tickers_openning('SI=F', ftickers, fticker_names)
hgf_df = tickers_openning('HG=F', ftickers, fticker_names)
plf_df = tickers_openning('PL=F', ftickers, fticker_names)

In [ ]:
model = ticker_co_df('GC=F', gcf_df)
print_model(model)

[*********************100%***********************]  1 of 1 completed
0.5748355263157895
[[595 106]
 [411 104]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_2996\2826252514.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.553453947368421, 0.602796052631579)


In [ ]:
model = ticker_co_df('SI=F', sif_df)
print_model(model)

[*********************100%***********************]  1 of 1 completed
0.6620065789473685
[[789   9]
 [402  16]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_2996\2826252514.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.640625, 0.6866776315789473)


In [ ]:
model = ticker_co_df('HG=F', hgf_df)
print_model(model)

[*********************100%***********************]  1 of 1 completed
0.5008223684210527
[[609   0]
 [607   0]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_2996\2826252514.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.4712171052631579, 0.5238486842105263)


In [ ]:
model = ticker_co_df('PL=F', plf_df)
print_model(model)

[*********************100%***********************]  1 of 1 completed
0.7179276315789473
[[806  74]
 [269  67]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_2996\2826252514.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.712171052631579, 0.7467105263157895)


# Question 4

For this question use the following [data](https://archive.ics.uci.edu/ml/datasets/credit+approval):


In [ ]:
credit = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None)

fn = {'+': 1, '-': 0}

X = credit.replace('?',0).iloc[:,[1,2,7,10,14]]
y = credit.iloc[:,15].map(lambda x: fn.get(x,0))

1. Split the data into training and test set.
2. Write different logistic regression models predicting y against X.
3. Construct [confusion matrices](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) on the test data set for these different models.
4. Analyze these models. Explain which model is the best model you have found.
5. Repeat Steps 1-4 several times. Does your best model stay as the best model? What should be the correct protocol to decide on the best model explaining the data?

In [ ]:
X = X.astype(float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

***

In [ ]:
model1 = LogisticRegression(max_iter=1500)
model1.fit(X_train, y_train)
model1.score(X_test, y_test)

0.8034682080924855

In [ ]:
y_predict = model1.predict(X_test)
confusion_matrix(y_test, y_predict)

array([[92,  8],
       [26, 47]], dtype=int64)

***

In [ ]:
df = pd.concat([X_train, y_train], axis=1)
df.columns = ['a','b','c','d','e','p']

In [ ]:
model2 = smf.logit('p ~ a + b + c + d + e', data=df).fit()
model2.summary()

Optimization terminated successfully.
         Current function value: 0.503675
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      p   No. Observations:                  517
Model:                          Logit   Df Residuals:                      511
Method:                           MLE   Df Model:                            5
Date:                Fri, 04 Nov 2022   Pseudo R-squ.:                  0.2686
Time:                        02:22:39   Log-Likelihood:                -260.40
converged:                       True   LL-Null:                       -356.03
Covariance Type:            nonrobust   LLR p-value:                 2.098e-39
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.7755      0.317     -5.605      0.000      -2.396      -1.155
a              0.0099      0.009      1.071      0.284      -0.008       0.028
b              0.0279      0.023      1.202      0.229      -0.018       0.073
c              0.1576      0.042      3.711      0.000       0.074       0.241
d              0.3628      0.052      6.996      0.000       0.261       0.464
e              0.0004      0.000      3.615      0.000       0.000       0.001
==============================================================================
"""

***

In [ ]:
model3 = sm.Logit(y_train, X_train).fit()
model3.summary()

Optimization terminated successfully.
         Current function value: 0.535929
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     15   No. Observations:                  517
Model:                          Logit   Df Residuals:                      512
Method:                           MLE   Df Model:                            4
Date:                Fri, 04 Nov 2022   Pseudo R-squ.:                  0.2218
Time:                        02:42:26   Log-Likelihood:                -277.08
converged:                       True   LL-Null:                       -356.03
Covariance Type:            nonrobust   LLR p-value:                 4.097e-33
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
1             -0.0358      0.005     -6.659      0.000      -0.046      -0.025
2             -0.0122      0.023     -0.539      0.590      -0.057       0.032
7              0.1632      0.041      3.953      0.000       0.082       0.244
10             0.3271      0.048      6.845      0.000       0.233       0.421
14             0.0003      0.000      3.277      0.001       0.000       0.001
==============================================================================
"""

In [384]:
y_predict3 = model3.predict(X_test)
y_predict3 = np.where(y_predict3 > 0.5, 1, 0)
confusion_matrix(y_test, y_predict3)

array([[90, 10],
       [30, 43]], dtype=int64)

***

In [ ]:
model4 = DecisionTreeRegressor()
model4.fit(X_train, y_train)
bootstrap(X, y, model4)

(-0.47497249724972557, 0.028609625668448935)

In [ ]:
y_predict4 = model4.predict(X_test)
confusion_matrix(y_test, y_predict4)

array([[89, 11],
       [ 4, 69]], dtype=int64)

***

In [ ]:
model5 = LinearRegression()
model5.fit(X_train, y_train)
bootstrap(X,y,model)

(-0.023122737665004944, 0.28493027263328663)

In [ ]:
y_predict5 = model5.predict(X_test)
y_predict5 = np.where(y_predict5 > 0.5, 1, 0)
confusion_matrix(y_test, y_predict5)

array([[95,  5],
       [34, 39]], dtype=int64)

***

Binary logistic regression model is the best model because y has 2 values (1 and 0). LogisticRegression() predicts y better than the others.